In [152]:
torch.setdefaulttensortype('torch.FloatTensor')

### Generate reference results using the OLD `IntegralSmartNorm`

In [153]:
require 'image'

nInputPlane = 3
nWindows = 4
h = 200
w = 200

referencePath = 'Checker/reference.t7'
if lfs.attributes(referencePath) then
    reference = torch.load(referencePath)
else
    reference = {}
    reference.image = image.scale(image.lena(), h, w)
    
    reference.xMin = torch.Tensor{-200, 35.5,  99, 0.0}
    reference.xMax = torch.Tensor{-195, 35.5, 100, 1.1}
    reference.yMin = torch.Tensor{  95, -1.0,  99,  -10}
    reference.yMax = torch.Tensor{ 100, -1.0, 100,   0}
    
    IntegralSmartNorm = nil
    debug.getregistry()['IntegralSmartNorm'] = nil 
    package.loaded['IntegralSmartNorm'] = nil
    require 'IntegralSmartNorm'
    
    int = IntegralSmartNorm(nWindows, h, w)
    int.exact = true
    int.smart = true
    for _, param in ipairs{'xMin', 'xMax', 'yMin', 'yMax'} do
        int[param]:copy(reference[param])
    end
    
    int:forward(reference.image)
    reference.outputNonNorm = int.outputNonNorm:clone()
    reference.outputOnes = torch.repeatTensor(int.outputOnes, nInputPlane, 1, 1)
    
    torch.manualSeed(666)
    reference.gradOutput = torch.rand(int.output:size())
    int:backward(reference.image, reference.gradOutput)
    for _, param in ipairs{'gradInput', 'gradXMin', 'gradXMax', 'gradYMin', 'gradYMax'} do
        reference[param] = int[param]
    end
    
--     torch.save(referencePath, reference)
end

### Check new `IntegralSmartNorm`'s output against the reference

In [154]:
IntegralSmartNorm = nil
debug.getregistry()['IntegralSmartNorm'] = nil 
package.loaded['IntegralSmartNorm'] = nil
require 'IntegralSmartNorm'

int = IntegralSmartNorm(nInputPlane, nWindows, h, w)
int.exact = true
int.smart = true
for _, param in ipairs{'xMin', 'xMax', 'yMin', 'yMax'} do
    int[param]:copy(torch.repeatTensor(reference[param], nInputPlane))
end

int:forward(reference.image)
print('outputOnes diff:', (reference.outputOnes - int.outputOnes):abs():mean())
print('outputNonNorm diff:', (reference.outputNonNorm - int.outputNonNorm):abs():max())

int:updateGradInput(reference.image, reference.gradOutput)
print('gradInput diff:', (reference.gradInput - int.gradInput):abs():max())

int:accGradParameters(reference.image, reference.gradOutput)
for _, param in ipairs{'gradInput', 'gradXMin', 'gradXMax', 'gradYMin', 'gradYMax'} do
--     print(param .. ' diff:', 
--         (reference[param] - int[param]:view(nInputPlane, nWindows):sum(1)):abs():max())
end

outputOnes diff:	0	


outputNonNorm diff:	0	


./IntegralSmartNorm.lua:559: attempt to call global 'flattenFirstDims' (a nil value)
stack traceback:
	./IntegralSmartNorm.lua:559: in function 'accGradParameters'
	[string "IntegralSmartNorm = nil..."]:20: in main chunk
	[C]: in function 'xpcall'
	...ubb/Programs/Torch/install/share/lua/5.1/itorch/main.lua:210: in function <...ubb/Programs/Torch/install/share/lua/5.1/itorch/main.lua:174>
	...ubb/Programs/Torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	.../Programs/Torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	.../Programs/Torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	.../Programs/Torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...ubb/Programs/Torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x00405e40: 

gradInput diff:	3.814697265625e-06	


In [167]:
torch.manualSeed(666)
local h,w = 2,2
img = torch.ones(1, h, w)

int = IntegralSmartNorm(1, 1, h, w)

int.exact = true
int.smart = false
crit = nn.MSECriterion()
target = torch.ones(1, h, w)

int.xMin[1] = 0.4
int.xMax[1] = 0.51
int.yMin[1] = 0
int.yMax[1] = 0

img[{1,1,1}] = 5
int:forward(img)
print(int.xMin[1])
print(int.xMax[1])
print(int.yMin[1])
print(int.yMax[1])

print('input')
print(img)
print('output')
print(int.output)
print('target')
print(target)
loss = crit:forward(int.output, target)
print('loss')
print(loss)
gradOutput = crit:updateGradInput(int.output, target)
print('gradOutput')
print(gradOutput)
int:updateGradInput(img, gradOutput)
print('backpropHelper params')
print(int.backpropHelper.xMin[1])
print(int.backpropHelper.xMax[1])
print(int.backpropHelper.yMin[1])
print(int.backpropHelper.yMax[1])
print('gradInput')
print(int.gradInput)

params = {}
loss = {}
deriv = {}
derivM = {}

local k = 1
local step = 0.05
local innerStep = 0.005

for param = -4,6,step do
    params[k] = param
    img[{1,1,1}] = param
    pred = int:forward(img)
    loss[k] = crit:forward(pred, target)
    
    int:updateGradInput(img, crit:updateGradInput(pred, target))
    derivM[k] = int.gradInput[{1,1,1}]
    
    img[{1,1,1}] = param + innerStep
    valFront = crit:forward(int:forward(img), target)
    img[{1,1,1}] = param - innerStep
    valBack = crit:forward(int:forward(img), target)
    
    deriv[k] = (valFront - valBack) / (2 * innerStep)
    
    k = k + 1
end

-- loss[#loss] = nil
-- params[#params] = nil
-- derivM[#derivM] = nil

0.40000000596046	
0.50999999046326	
0	
0	
input	
(1,.,.) = 
  5  1
  1  1
[torch.FloatTensor of size 1x2x2]

output	
(1,.,.) = 
  3.5100  1.1100
  1.0000  1.0000
[torch.FloatTensor of size 1x2x2]

target	
(1,.,.) = 
  1  1
  1  1
[torch.FloatTensor of size 1x2x2]

loss	
1.5780500173569	
gradOutput	
(1,.,.) = 
  1.2550  0.0550
  0.0000  0.0000
[torch.FloatTensor of size 1x2x2]

backpropHelper params	
-0.50999999046326	
-0.40000000596046	
-0	
-0	
gradInput	
(1,.,.) = 
  1.2550  0.0550
  0.6400  0.0280
[torch.FloatTensor of size 1x2x2]



In [168]:
require 'gnuplot'

gnuplot.plot(
    {'Loss', torch.Tensor(params), torch.Tensor(loss), '-'},
    {'Diff', torch.Tensor(params), torch.Tensor(deriv), '-'},
    {'Manual', torch.Tensor(params), torch.Tensor(derivM), '-'}
)
gnuplot.grid(true)